In [2]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from collections import Counter
from itertools import product

In [3]:
# teste rapido de calculo de embeddings com chatGPT
corpus = [
    "o gato comeu o rato",
    "o cachorro correu atrás do gato",
    "o pássaro voou para longe",
    "o peixe nadou no rio"
]

In [4]:
# Tokenização simples
tokenized_corpus = [sentence.split() for sentence in corpus]
tokenized_corpus

[['o', 'gato', 'comeu', 'o', 'rato'],
 ['o', 'cachorro', 'correu', 'atrás', 'do', 'gato'],
 ['o', 'pássaro', 'voou', 'para', 'longe'],
 ['o', 'peixe', 'nadou', 'no', 'rio']]

In [6]:
# Criar vocabulário
vocab = set(word for sentence in tokenized_corpus for word in sentence)
vocab = list(vocab)
vocab_size = len(vocab)

# Mapeamento palavra <-> índice
word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for word, i in word_to_index.items()}

from pprint import pprint

pprint(word_to_index)

{'atrás': 9,
 'cachorro': 11,
 'comeu': 0,
 'correu': 1,
 'do': 12,
 'gato': 14,
 'longe': 10,
 'nadou': 7,
 'no': 3,
 'o': 8,
 'para': 2,
 'peixe': 5,
 'pássaro': 6,
 'rato': 13,
 'rio': 4,
 'voou': 15}


In [ ]:
# Inicializar matriz de coocorrência
cooccurrence_matrix = np.zeros((vocab_size, vocab_size), dtype=np.float32) # Da pra interpretar como um grafo

# Janela de contexto
window_size = 2

# Preencher matriz de coocorrência
for sentence in tokenized_corpus:
    for i, word in enumerate(sentence):
        word_idx = word_to_index[word]
        # Determinar janela de contexto
        start = max(i - window_size, 0)
        end = min(i + window_size + 1, len(sentence))
        # Adicionar coocorrências
        for j in range(start, end):
            if i != j:  # Ignorar a palavra atual
                context_idx = word_to_index[sentence[j]]
                cooccurrence_matrix[word_idx, context_idx] += 1

print("Matriz de Coocorrência:")
pprint(cooccurrence_matrix)


Matriz de Coocorrência:
array([[0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 2., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.,

In [ ]:
# Reduzir dimensão para embeddings
embedding_dim = 4

svd = TruncatedSVD(n_components=embedding_dim)
word_embeddings = svd.fit_transform(cooccurrence_matrix)

print("Embeddings Calculados:")
for word, embedding in zip(vocab, word_embeddings):
    pprint(f"{word}: {embedding}") # Lembre que cada linha do embedding representa uma palavra


Embeddings Calculados:
'comeu: [ 1.7673157e+00 -8.6055112e-01 -1.9129515e-02  3.4633558e-08]'
'correu: [ 1.1843683e+00 -8.3020598e-01  8.3977550e-01 -7.7918958e-07]'
'para: [ 0.4168095   0.32144898 -0.76711947  1.0095162 ]'
'no: [ 0.4168095   0.32144862 -0.7671209  -1.0095146 ]'
'rio: [ 0.27068844  0.08879717 -0.5862039  -0.78820515]'
'peixe: [ 0.8603785  -0.6085834  -0.67432    -0.78820497]'
'pássaro: [ 0.8603785  -0.608583   -0.67431855  0.7882065 ]'
'nadou: [ 0.9063503  -0.6425294  -0.83159584 -1.0095142 ]'
'o: [ 2.8824861e+00  2.5216212e+00 -2.4031439e-01 -2.0861626e-07]'
'atrás: [ 1.0213596e+00  6.9658637e-01  1.0447264e+00 -1.2479941e-06]'
'longe: [ 0.27068844  0.08879752 -0.58620244  0.78820634]'
'cachorro: [ 1.0409317e+00 -6.6042697e-01  6.0287690e-01 -4.4111221e-07]'
'do: [ 8.4457457e-01  4.4411576e-01  8.8297749e-01 -8.7934313e-07]'
'rato: [ 9.5124280e-01 -4.5938718e-01 -9.5130488e-02  6.4552296e-08]'
'gato: [ 1.9226600e+00 -1.4722406e+00  5.2358735e-01 -4.3179170e-07]'
'voou